In [ ]:
import pandas as pd
import boto3
import json

## Dataset : downloading - preprocessing - uploading
First enter the [competition](https://www.kaggle.com/c/forest-cover-type-prediction) and download the dataset.

In [ ]:
raw_data = pd.read_csv('data/train.csv', index_col=0)
raw_data.head(2)

As there are 55 columns in the train.csv I would suggest you to create datasource from [aws website](https://aws.amazon.com/) itself, as it's interface would become easy to interact with rather than writing codes.

To solve the whole problem right there on the website, you can refer [this](https://github.com/shravankumar9892/bank-marketing/blob/master/README.md).

1. First save the dataset in a S3 Bucket
2. Create the datasource

![Creating Data Source](images/datasourcecreate.png)

If it asks -> Does the first line in your CSV contain the column names? click on **yes**

**Now the most important part comes, because of which we preferred the aws website interface**
1. Convert all Soil_Typex features into BINARY
2. Convert all Wilderness_Areax into BINARY
3. Convert Cover_Type into CATEGORICAL

If it asks -> Do you plan to use this dataset to create or evaluate an ML model? Click **yes**
And then Select Cover_Type as value you want to predict.

In [ ]:
client = boto3.client('machinelearning')

__ = client.create_ml_model(
    MLModelId='kaggleisthebest',
    MLModelName='covertypemulticlass',
    MLModelType='MULTICLASS',   # Amazon ML has 3 types of model types: BINARY | MULTICLASS | REGRESSION
    TrainingDataSourceId='ds-ooCQMKjVxXp' # Replace it with your corresponding DataSource ID.
)

In [ ]:
# To create batch predictions on the test data we again need to create a datasource
# Remember the test.csv file has size of 72.6 mb, so it make time to create the datasource 
# But this time while creating the dataset, if they ask --> Do you plan to use this dataset to create or evaluate an ML model?
# You should click No

___ = client.create_batch_prediction(
    BatchPredictionId='batchpredictionforest,
    BatchPredictionName='predictresultsofforest',
    MLModelId='kaggleisthebest',
    BatchPredictionDataSourceId='ds-siouUHDY66y',
    OutputUri='s3://forestcoverkaggle/'
)

# OutputUri specifies in which S3 bucket directory shall the prediction folder be placed.

Once the predictions folder is generated in your S3 Bucket, download it as a csv.

Now to submit it to the Kaggle competition, we need to modify the predictions.csv

In [ ]:
predictions = pd.read_csv('data/predictions.csv')
output = pd.read_csv('data/sampleSubmission.csv')

output['Cover_Type'] = predictions.idxmax(axis = 1)
output.to_csv('prediction.csv', index = False)

The output file delibrately had the predictive score, to make it more flexible. We did nothing but, taking maximum of each row from the predictions dataframe and then putting it in the output dataframe 